# OpenCV Face Recognition

This script uses OpenCV and Dlib's pre-trained models to recognize faces in real-time through a webcam feed.  
The program captures video from the webcam, detects faces, and then compares them with known faces stored in a local dataset.  
If a match is found, the name of the recognized individual is displayed. If no match is found, it shows "Unknown."

## How it works:
- Captures frames from the webcam.
- Detects faces using Dlib's face detector.
- Encodes the faces and compares them with a known dataset of face encodings.
- Displays the name of the recognized person or "Unknown" if no match is found.
- Draws a rectangle around the detected faces and labels them with the name.

Press 'q' to exit the webcam feed.

## Prerequisites:
1. Pre-trained models (Dlib's shape predictor and face recognition models) should be downloaded and placed in the `models/` folder.
2. Known faces (images) should be stored in the `dataset/` folder.


## Kütüphaneleri İçeri Aktar

In [1]:
import cv2
import dlib
import os
import numpy as np

## Yüz tanıma modeli yükleme

In [2]:
# Load the pre-trained face landmark detector from Dlib
# Shape predictor is used to detect facial landmarks (e.g. eyes, nose, mouth)
detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")  # Landmark modeli

# Load the pre-trained face recognition model from Dlib
facerec = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")  # Yüz tanıma modeli

# Kişileri ve yüz özelliklerini saklayacak dictionary
known_faces = {}
dataset_path = "dataset"

## Dataset klasöründeki yüzleri tanı

In [3]:
# Dataset'teki her yüzü tanıt
for filename in os.listdir(dataset_path):
    name, ext = os.path.splitext(filename)
    img_path = os.path.join(dataset_path, filename)
    
    img = cv2.imread(img_path)
     # Convert the frame to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the frame
    faces = detector(gray)
    if len(faces) > 0:
        shape = sp(img, faces[0])  # İlk yüzü al
        face_descriptor = facerec.compute_face_descriptor(img, shape)
        known_faces[name] = np.array(face_descriptor)

In [6]:
# open camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        shape = sp(frame, face)
        face_descriptor = facerec.compute_face_descriptor(frame, shape)
        face_descriptor = np.array(face_descriptor)

        # Find the name of the matched face with the minimum distance
        min_dist = float("inf")
        identity = "Unknown"

        for name, known_face in known_faces.items():
            dist = np.linalg.norm(known_face - face_descriptor)
            if dist < 0.4:  # Threshold for recognition accuracy
                min_dist = dist
                identity = name

        # İsmi ekrana yaz
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, identity, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):  # Break the loop if the user presses the 'q' key
        break

# Release the capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()